In [3]:
import xlwings as xw
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Connect to the open workbook "ES RTD.xlsx"
wb = xw.Book('ES RTD.xlsx')  # Adjust to the correct file path

# Access the specific sheet "ES SPX Difference" and retrieve the value from cell C6
ws_diff = wb.sheets['ES SPX Difference']
ESPXdiff = ws_diff.range('C6').value  # Now dynamically pulls the value from C6

# Access the specific sheet "ES SPX Difference" and retrieve the value from cell E8
ws_diff = wb.sheets['SPY SPX Difference']
SPXSPYdiff = ws_diff.range('E8').value  # Now dynamically pulls the value from E8

# Extract the SPX spot price and time-related data from 'SPX RTD'
ws_spx = wb.sheets['SPX RTD']

# Extract dynamic date and time from cell A1
date_info = ws_spx.range('A1').value
date_part = date_info.split(' on ')[1].split(' ')[0]

# Extract SPX Spot Price from A4
SPX_spot = ws_spx.range('A4').value

# Define the start row
start_row = 10

# Automatically detect the last row with useful data (assuming column A reliably contains data)
row = start_row
while True:
    value = ws_spx.range(f'C{row}').value
    if not value:  # Stop when an empty cell is found
        end_row = row - 1  # Set the end row to the last non-empty row
        break
    row += 1

print(f"Start row: {start_row}, End row: {end_row}")


# Initialize lists to store strikes and gamma exposures for Calls and Puts
strikes = []
call_gamma_exposure_list = []
put_gamma_exposure_list = []
call_volumes = []
put_volumes = []

# Loop through each row and extract relevant data for Calls and Puts
for row in range(start_row, end_row + 1):
    strike_price = ws_spx.range(f'Q{row}').value
    call_gamma = ws_spx.range(f'I{row}').value
    call_open_int = ws_spx.range(f'K{row}').value
    put_gamma = ws_spx.range(f'AB{row}').value
    put_open_int = ws_spx.range(f'AD{row}').value
    call_volume = ws_spx.range(f'G{row}').value  # Volume for Calls
    put_volume = ws_spx.range(f'Z{row}').value  # Volume for Puts
    
    if call_gamma and call_open_int:
        call_gamma_exposure = call_gamma * call_open_int * (SPX_spot)**2 * 0.01
    else:
        call_gamma_exposure = 0
    
    if put_gamma and put_open_int:
        put_gamma_exposure = put_gamma * put_open_int * (SPX_spot)**2 * 0.01 * -1
    else:
        put_gamma_exposure = 0
    
    strikes.append(strike_price)
    call_gamma_exposure_list.append(call_gamma_exposure)
    put_gamma_exposure_list.append(put_gamma_exposure)
    call_volumes.append(call_volume)
    put_volumes.append(put_volume)

# Define the ±350 range from the SPX Spot Price
minStrike = SPX_spot - 350
maxStrike = SPX_spot + 350

# Create a DataFrame
df = pd.DataFrame({
    'Strike': strikes,
    'Call Gamma Exposure': call_gamma_exposure_list,
    'Put Gamma Exposure': put_gamma_exposure_list,
    'Call Volume': call_volumes,
    'Put Volume': put_volumes
})

# Filter the DataFrame to only show strikes within the ±350 range of SPX Spot Price
df_filtered = df[(df['Strike'] >= minStrike) & (df['Strike'] <= maxStrike)].copy()  # Explicitly create a copy

# Use .loc[] to avoid SettingWithCopyWarning
df_filtered.loc[:, 'Total Gamma Exposure'] = df_filtered['Call Gamma Exposure'] + df_filtered['Put Gamma Exposure']
df_filtered.loc[:, 'Cumulative Call Gamma'] = df_filtered['Call Gamma Exposure'].cumsum()
df_filtered.loc[:, 'Cumulative Put Gamma'] = df_filtered['Put Gamma Exposure'].cumsum()

# ---- Find Gamma Flip Point ----
totalGamma_filtered = df_filtered['Total Gamma Exposure'].values
levels_filtered = df_filtered['Strike'].values
zeroCrossIdx = np.where(np.diff(np.sign(totalGamma_filtered)))[0]

# Check if there's any zero-crossing index
if zeroCrossIdx.size > 0:
    # Store potential flip points
    flip_points = []
    for idx in zeroCrossIdx:
        negGamma = totalGamma_filtered[idx]
        posGamma = totalGamma_filtered[idx + 1]
        negStrike = levels_filtered[idx]
        posStrike = levels_filtered[idx + 1]

        # Linear interpolation to find the exact Gamma Flip point
        interpolated_flip = posStrike - ((posStrike - negStrike) * posGamma / (posGamma - negGamma))
        flip_points.append(interpolated_flip)

    # If multiple flip points, find the one closest to the SPX Spot Price
    flip_points = np.array(flip_points)
    closest_flip_idx = np.argmin(np.abs(flip_points - SPX_spot))
    zeroGamma = flip_points[closest_flip_idx]
else:
    zeroGamma = None
    print("Warning: No Gamma Flip Point found. Skipping the Gamma Flip line in the chart.")

# ---- Chart Creation Section ----

# Initialize the figure
fig = go.Figure()

# Create the bar chart (Gamma Exposure)
bar_chart_call = go.Bar(x=df_filtered['Strike'], y=df_filtered['Call Gamma Exposure'], name='Call Gamma', marker_color='blue')
bar_chart_put = go.Bar(x=df_filtered['Strike'], y=df_filtered['Put Gamma Exposure'], name='Put Gamma', marker_color='red')
bar_chart_total = go.Bar(x=df_filtered['Strike'], y=df_filtered['Total Gamma Exposure'], name='Total Gamma', marker_color='green')

# Create the scatter plot (Gamma Exposure vs. Strike Price)
scatter_call = go.Scatter(x=df_filtered['Strike'], y=df_filtered['Call Gamma Exposure'], mode='markers+lines', name='Call Gamma', marker=dict(color='blue', size=8))
scatter_put = go.Scatter(x=df_filtered['Strike'], y=df_filtered['Put Gamma Exposure'], mode='markers+lines', name='Put Gamma', marker=dict(color='red', size=8))

# Create cumulative gamma exposure scatter plots
cumulative_call = go.Scatter(x=df_filtered['Strike'], y=df_filtered['Cumulative Call Gamma'], mode='lines', name='Cumulative Call Gamma', line=dict(color='blue', width=2))
cumulative_put = go.Scatter(x=df_filtered['Strike'], y=df_filtered['Cumulative Put Gamma'], mode='lines', name='Cumulative Put Gamma', line=dict(color='red', width=2))

# Create the volume distribution chart (Volume by Strike Price)
volume_call = go.Bar(x=df_filtered['Strike'], y=df_filtered['Call Volume'], name='Call Volume', marker_color='blue')
volume_put = go.Bar(x=df_filtered['Strike'], y=df_filtered['Put Volume'], name='Put Volume', marker_color='red')

# Add a vertical line for SPX Spot Price
y_min = min(df_filtered['Call Gamma Exposure'].min(), df_filtered['Put Gamma Exposure'].min())
y_max = max(df_filtered['Call Gamma Exposure'].max(), df_filtered['Put Gamma Exposure'].max())
spot_price_line = go.Scatter(
    x=[SPX_spot, SPX_spot],  # Constant x (SPX Spot Price)
    y=[y_min, y_max],  # Updated y-range
    mode='lines',
    name=f'SPX Spot Price: {SPX_spot}',
    line=dict(color='black', width=3, dash='solid')  # Style the line
)

# ---- Add Gamma Flip line to all charts ----
if zeroGamma:
    gamma_flip_line = go.Scatter(
        x=[zeroGamma, zeroGamma],  # Constant x (Gamma Flip Strike)
        y=[y_min, y_max],  # Updated y-range
        mode='lines',
        name=f'Gamma Flip: {zeroGamma}',
        line=dict(color='purple', width=3, dash='dash')  # Style the line
    )

# Add traces to the figure
fig.add_traces([bar_chart_call, bar_chart_put, scatter_call, scatter_put, cumulative_call, cumulative_put, volume_call, volume_put, bar_chart_total, spot_price_line])

# Add Gamma Flip line to all charts
if zeroGamma:
    fig.add_trace(gamma_flip_line)

# Define the dropdown menu options
dropdown_buttons = [
    dict(label='Gamma Exposure (Bar)',
         method='update',
         args=[{'visible': [True, True, False, False, False, False, False, False, False, True, True]},
               {'title': f'Gamma Exposure by Strike Price (SPX) - {date_part}'}]),
    dict(label='Volume Distribution',
         method='update',
         args=[{'visible': [False, False, False, False, False, False, True, True, False, False, False]}, 
               {'title': f'Volume Distribution by Strike Price (SPX) - {date_part}'}]),
    dict(label='Gamma Exposure (Scatter)',
         method='update',
         args=[{'visible': [False, False, True, True, False, False, False, False, False, True, True]},
               {'title': f'Gamma Exposure vs Strike Price (SPX) - {date_part}'}]),
    dict(label='Cumulative Gamma Exposure',
         method='update',
         args=[{'visible': [False, False, False, False, True, True, False, False, False, False, False]},
               {'title': f'Cumulative Gamma Exposure (SPX) - {date_part}'}]),
    dict(label='Total Gamma Exposure',
         method='update',
         args=[{'visible': [False, False, False, False, False, False, False, False, True, True, True]},
               {'title': f'Total Gamma Exposure by Strike Price (SPX) - {date_part}'}])
]

# Initialize all traces to be invisible except the default (Gamma Exposure bar chart)
fig.update_traces(visible=False)
fig.data[0].visible = True  # Call Gamma (Bar)
fig.data[1].visible = True  # Put Gamma (Bar)
fig.data[9].visible = True  # Spot Price Line
if zeroGamma:
    fig.data[-1].visible = True  # Gamma Flip line

# Add dropdown menu
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=dropdown_buttons,
        x=1.15,  # Positioning the dropdown
        y=1.15,
        xanchor='right'
    )]
)

# Update layout
fig.update_layout(
    title=f'Gamma Exposure by Strike Price (SPX) - {date_part}',
    xaxis_title='Strike Price',
    yaxis_title='Gamma Exposure',
    barmode='group',
    xaxis_tickangle=-45
)

# Show the figure with dropdown
fig.show()

# ---- Data Saving Section ----

# Save top 5 and lowest 5 total gamma exposure levels in a separate workbook
top_5_gamma = df_filtered.nlargest(5, 'Total Gamma Exposure')
lowest_5_gamma = df_filtered.nsmallest(5, 'Total Gamma Exposure')

# Define the Gamma Level for the top and lowest 5
top_5_gamma['GammaLvl'] = ['Call Gamma 1', 'Call Gamma 2', 'Call Gamma 3', 'Call Gamma 4', 'Call Gamma 5']
lowest_5_gamma['GammaLvl'] = ['Put Gamma 1', 'Put Gamma 2', 'Put Gamma 3', 'Put Gamma 4', 'Put Gamma 5']

# Use .loc[] to avoid SettingWithCopyWarning for Theo ES Price calculation
top_5_gamma = top_5_gamma.copy()  # Make sure we work with a copy to avoid warnings
lowest_5_gamma = lowest_5_gamma.copy()

top_5_gamma.loc[:, 'Theo ES Price'] = (top_5_gamma['Strike'] * ESPXdiff + top_5_gamma['Strike']).round(2)
top_5_gamma.loc[:, 'Theo ES Price'] = (top_5_gamma['Theo ES Price'] / 0.25).round() * 0.25

lowest_5_gamma.loc[:, 'Theo ES Price'] = (lowest_5_gamma['Strike'] * ESPXdiff + lowest_5_gamma['Strike']).round(2)
lowest_5_gamma.loc[:, 'Theo ES Price'] = (lowest_5_gamma['Theo ES Price'] / 0.25).round() * 0.25

# Multiply the 'Total Gamma Exposure' by 0.000001
top_5_gamma.loc[:, 'Total Gamma Exposure'] = top_5_gamma['Total Gamma Exposure'] * 0.000001
lowest_5_gamma.loc[:, 'Total Gamma Exposure'] = lowest_5_gamma['Total Gamma Exposure'] * 0.000001

# Modify the Gamma Flip calculation by applying the formula (Gamma Flip * ESPXdiff + Gamma Flip)
adjusted_gamma_flip = zeroGamma * ESPXdiff + zeroGamma

# Round the adjusted Gamma Flip to the nearest 0.25
adjusted_gamma_flip = (adjusted_gamma_flip / 0.25).round() * 0.25

# Ensure the adjusted and rounded Gamma Flip value is applied to the entire column
top_5_gamma.loc[:, 'Gamma Flip'] = adjusted_gamma_flip  # Apply the adjusted and rounded Gamma Flip value to all rows
lowest_5_gamma.loc[:, 'Gamma Flip'] = adjusted_gamma_flip  # Apply the adjusted and rounded Gamma Flip value to all rows

# Select only the relevant columns for top 5 and lowest 5
top_5_gamma = top_5_gamma[['Strike', 'Total Gamma Exposure', 'GammaLvl', 'Theo ES Price', 'Gamma Flip']]
lowest_5_gamma = lowest_5_gamma[['Strike', 'Total Gamma Exposure', 'GammaLvl', 'Theo ES Price', 'Gamma Flip']]

# Concatenate the top 5 and lowest 5 data into a single dataframe
combined_df = pd.concat([top_5_gamma, lowest_5_gamma], ignore_index=True)

# Rename the columns to match your desired output format
combined_df.columns = ['StrikePrice', 'TotalGamma', 'GammaLvl', 'Theo ES Price', 'Gamma Flip']

# Save the dataframe as a CSV file
output_csv_path = r'C:\Users\Chris\OneDrive\Documents\NinjaTrader 8\bin\Custom\Top_Lowest_5_Gamma_Exposures.csv'
combined_df.to_csv(output_csv_path, index=False)

print("Top 5 & lowest 5 gamma exposures saved to CSV with consistent Gamma Flip value and Theo ES Price.")

# ---- Save Theo SPY Price to a separate CSV ----
# Calculate Theo SPY Price for top 5 and lowest 5 gamma
top_5_spy = top_5_gamma[['Strike', 'Total Gamma Exposure']].copy()  # Include Total Gamma Exposure
lowest_5_spy = lowest_5_gamma[['Strike', 'Total Gamma Exposure']].copy()  # Include Total Gamma Exposure

top_5_spy['Theo SPY Price'] = (top_5_spy['Strike'] * (1 - SPXSPYdiff)).round(2)
lowest_5_spy['Theo SPY Price'] = (lowest_5_spy['Strike'] * (1 - SPXSPYdiff)).round(2)


# Concatenate the top 5 and lowest 5 SPY data into a single dataframe, including Total Gamma Exposure
spy_df = pd.concat([top_5_spy, lowest_5_spy], ignore_index=True)

# Save the Theo SPY Price dataframe with Total Gamma Exposure as a separate CSV file
output_spy_csv_path = r'C:\Users\Chris\OneDrive\Documents\NinjaTrader 8\bin\Custom\Theo_SPY_Price.csv'
spy_df.to_csv(output_spy_csv_path, index=False)

print("Theo SPY Price with Total Gamma Exposure saved to a separate CSV.")

Start row: 10, End row: 267


Top 5 & lowest 5 gamma exposures saved to CSV with consistent Gamma Flip value and Theo ES Price.
Theo SPY Price with Total Gamma Exposure saved to a separate CSV.


In [4]:
import pandas as pd
import plotly.graph_objects as go

# Load the previously generated SPY Theo Price CSV file
spy_theo_df = pd.read_csv(r'C:\Users\Chris\OneDrive\Documents\NinjaTrader 8\bin\Custom\Theo_SPY_Price.csv')

# Sort the DataFrame by 'Total Gamma Exposure' in descending order
spy_theo_df = spy_theo_df.sort_values(by='Total Gamma Exposure', ascending=False)

# Assign colors: green shades for top 5, red shades for bottom 5, and blue for the rest
colors = ['blue'] * len(spy_theo_df)  # Initialize all bars as blue

# Define shades of green for top 5
green_shades = ['#006400', '#228B22', '#32CD32', '#7CFC00', '#ADFF2F']

# Define shades of red for bottom 5 (darkest red for the lowest gamma)
red_shades = ['#8B0000', '#B22222', '#DC143C', '#FF4500', '#FF6347'][::-1]  # Reversed order for red shades

# Assign green shades to the top 5 gamma levels
colors[:5] = green_shades

# Assign reversed red shades to the bottom 5 gamma levels (darkest red for the lowest)
colors[-5:] = red_shades

# Create a horizontal bar chart
fig = go.Figure(go.Bar(
    x=spy_theo_df['Total Gamma Exposure'],
    y=spy_theo_df['Theo SPY Price'],
    orientation='h',
    marker_color=colors  # Apply the colors to the bars
))

# Reverse the Y-axis to show highest Total Gamma Exposure at the top
fig.update_layout(
    title='SPY Theo Price Levels Measured by Total Gamma Exposure',
    xaxis_title='Total Gamma Exposure',
    yaxis_title='Theo SPY Price',
    yaxis=dict(categoryorder='total ascending')  # Ensures the Y-axis is ordered with highest gamma at the top
)

# Show the plot
fig.show()
